In [1]:
import torch # pytorch basic package
from torch import nn # neural net 
from torch.utils.data import DataLoader, Dataset # to work with data
from torchvision import datasets # built-in data
from torchvision.transforms import ToTensor # to convert nparrays/images into pytorch tensors
from torchvision import models
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from nn_utils import train, test

In [2]:
torch.manual_seed(0) # for reproducibility
device = "cuda" if torch.cuda.is_available() else "cpu" # set GPU

## The ImageNet dataset
Let's experiment with a new dataset with many more images and more classes:

In [3]:
# Download training data from open datasets.
training_data = datasets.CIFAR100(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.CIFAR100(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

trainloader = DataLoader(training_data, batch_size=8, shuffle= True)
testloader = DataLoader(test_data, batch_size=8, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
resnet_model = models.resnet50(weights=None)

num_classes = 100

print(resnet_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
resnet_model.fc = nn.Linear(in_features = resnet_model.fc.in_features, out_features = num_classes)
resnet_model = resnet_model.to(device)

print(resnet_model)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet_model.parameters(), lr=0.001, momentum=0.9)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
epochs = 10
train_losses = []
test_losses = []
for t in range(epochs):
    print(f'EPOCH {t} --------------------')
    train_loss = train(trainloader, resnet_model, loss, optimizer, 1000, device)
    test_loss = test(t, testloader, resnet_model, loss, device)

    train_losses.append(train_loss)
    test_losses.append(test_loss)

print("Done!")

EPOCH 0 --------------------
8/50000 : current_loss = 4.616127014160156
8008/50000 : current_loss = 5.035161972045898
16008/50000 : current_loss = 4.340090751647949
24008/50000 : current_loss = 5.0845561027526855
32008/50000 : current_loss = 3.875854969024658
40008/50000 : current_loss = 4.223443984985352
48008/50000 : current_loss = 4.272512912750244
End of epoch 1
 Accuracy: 7.69%, Average loss: 5.0368

EPOCH 1 --------------------
8/50000 : current_loss = 4.6453118324279785
8008/50000 : current_loss = 4.495233058929443
16008/50000 : current_loss = 3.55768084526062
24008/50000 : current_loss = 4.406271934509277
32008/50000 : current_loss = 3.8818416595458984
40008/50000 : current_loss = 4.32372522354126
48008/50000 : current_loss = 4.75855827331543
End of epoch 2
 Accuracy: 9.63%, Average loss: 6.3387

EPOCH 2 --------------------
8/50000 : current_loss = 3.8852767944335938
8008/50000 : current_loss = 3.448747158050537
16008/50000 : current_loss = 3.8662703037261963
24008/50000 : cur

In [5]:
resnet_model_pretrained = models.resnet50(weights="IMAGENET1K_V1")
resnet_model_pretrained.fc = nn.Linear(in_features = resnet_model_pretrained.fc.in_features, out_features = num_classes)
resnet_model_pretrained = resnet_model_pretrained.to(device)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet_model_pretrained.parameters(), lr=0.001, momentum=0.9)

epochs = 15
train_losses = []
test_losses = []
for t in range(epochs):
    print(f'EPOCH {t} --------------------')
    train_loss = train(trainloader, resnet_model_pretrained, loss, optimizer, 1000, device)
    test_loss = test(t, testloader, resnet_model_pretrained, loss, device)

    train_losses.append(train_loss)
    test_losses.append(test_loss)

print("Done!")

EPOCH 0 --------------------
8/50000 : current_loss = 4.557442665100098
8008/50000 : current_loss = 3.55440616607666
16008/50000 : current_loss = 3.827915668487549
24008/50000 : current_loss = 3.508331298828125
32008/50000 : current_loss = 4.535471439361572
40008/50000 : current_loss = 4.321124076843262
48008/50000 : current_loss = 3.790506601333618
End of epoch 1
 Accuracy: 18.41%, Average loss: 3.4049

EPOCH 1 --------------------
8/50000 : current_loss = 3.1941704750061035
8008/50000 : current_loss = 4.070655822753906
16008/50000 : current_loss = 2.7144856452941895
24008/50000 : current_loss = 3.192312240600586
32008/50000 : current_loss = 2.616659164428711
40008/50000 : current_loss = 4.739078044891357
48008/50000 : current_loss = 3.6653215885162354
End of epoch 2
 Accuracy: 13.49%, Average loss: 5.1949

EPOCH 2 --------------------
8/50000 : current_loss = 3.35467529296875
8008/50000 : current_loss = 3.2602334022521973
16008/50000 : current_loss = 3.1745383739471436
24008/50000 : 

KeyboardInterrupt: 

## AUGMENTATIONS

In [ ]:
import torchvision.transforms as transforms

In [ ]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(),
    transforms.GaussianBlur(kernel_size=3)
    #transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

In [ ]:
# Download test data from open datasets.
test_data = datasets.CIFAR100(
    root="data",
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

# Download training data from open datasets.
training_data = datasets.CIFAR100(
    root="data",
    train=True,
    download=True,
    transform=transform_train,
)

trainloader = DataLoader(training_data, batch_size=8, shuffle=True)
testloader = DataLoader(test_data, batch_size=8, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
resnet_model_pretrained = models.resnet50(weights="IMAGENET1K_V1")
resnet_model_pretrained.fc = nn.Linear(in_features = resnet_model_pretrained.fc.in_features, out_features = num_classes)
resnet_model_pretrained = resnet_model_pretrained.to(device)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet_model_pretrained.parameters(), lr=0.001, momentum=0.9)

epochs = 15
train_losses = []
test_losses = []
for t in range(epochs):
    print(f'EPOCH {t} --------------------')
    train_loss = train(trainloader, resnet_model_pretrained, loss, optimizer, 1000, device)
    test_loss = test(t, testloader, resnet_model_pretrained, loss, device)

    train_losses.append(train_loss)
    test_losses.append(test_loss)

print("Done!")

EPOCH 0 --------------------
8/50000 : current_loss = 4.952907085418701
8008/50000 : current_loss = 4.794363021850586
16008/50000 : current_loss = 3.801426410675049
24008/50000 : current_loss = 3.7443599700927734
32008/50000 : current_loss = 2.7729427814483643
40008/50000 : current_loss = 2.5280988216400146
48008/50000 : current_loss = 3.2149760723114014
End of epoch 1
 Accuracy: 29.03%, Average loss: 2.9658

EPOCH 1 --------------------
8/50000 : current_loss = 2.75014066696167
8008/50000 : current_loss = 3.589860200881958
16008/50000 : current_loss = 3.3368775844573975
24008/50000 : current_loss = 2.592991590499878
32008/50000 : current_loss = 2.6291942596435547
40008/50000 : current_loss = 4.089916706085205
48008/50000 : current_loss = 2.8373372554779053
End of epoch 2
 Accuracy: 36.24%, Average loss: 2.6958

EPOCH 2 --------------------
8/50000 : current_loss = 3.2689626216888428
8008/50000 : current_loss = 4.848925590515137
16008/50000 : current_loss = 2.4978599548339844
24008/500